In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] ="-1"
import torch
import numpy as np
import torch.nn as nn
# import torch.nn.functional as F
# import torchvision

#from keras.preprocessing.sequence import pad_sequences
#from keras.utils import to_categorical

from tqdm import tqdm_notebook as tqdm

import json
import nltk
from nltk import word_tokenize

from torch.utils.data.dataloader import default_collate


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
DIC_word_index = json.load(open("vocab.json", "r", encoding='utf-8'))
DIC_index_word = {index:word for word, index in DIC_word_index.items()}
word_vectors = np.load("wv_matrix100d.npy")

In [2]:
# if you don't have money, use this cell

# device = torch.device("cpu")
# print(device)
# DIC_word_index = json.load(open("vocab.json", "r", encoding='utf-8'))
# DIC_index_word = {index:word for word, index in DIC_word_index.items()}
# word_vectors = np.load("wv_matrix100d.npy")

cpu


In [3]:
sent_len = 5

In [4]:
class InferenceAttModel(torch.nn.Module):
    def __init__(self, latent_dim, seq_length, embed_dim, word_vectors):
        super(InferenceAttModel, self).__init__()
        
        self.hidden_dim = latent_dim        
        self.seq_length = seq_length
        self.embed_dim = embed_dim
        
#         self.embedding = torch.nn.Embedding(len(word_vectors), embed_dim)
#         self.embedding.load_state_dict({'weight': word_vectors})
#         self.embedding.weight.requires_grad = False
        self.embedding = torch.nn.Embedding.from_pretrained(torch.from_numpy(word_vectors), freeze=True)
#        self.embedding = torch.nn.Embedding(len(word_vectors), embed_dim)
    
        self.soft = torch.nn.Softmax(-2)
        
        self.RNN1 = torch.nn.LSTM(self.embed_dim, latent_dim,  num_layers=2, batch_first = True)
        self.RNN2 = torch.nn.LSTM(self.embed_dim, latent_dim,  num_layers=2, batch_first = True)
        
        self.Projection_layer = torch.nn.Sequential(
            torch.nn.Linear(2 * latent_dim, len(word_vectors)),
        )
        
        self.trainable_W = torch.nn.Sequential(
            torch.nn.Linear(latent_dim ,  latent_dim),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(p=0.2),
        )
    def encoder_RNN(self, e_x):
        e_y, hiddens = self.RNN1(e_x)
        
        return e_y, hiddens
    
    def decoder_RNN(self, d_x, hiddens):
        d_y, hiddens2 = self.RNN2(d_x, hiddens)
        return d_y, hiddens2
    
    def forward(self, e_x, ):
        e_x = self.embedding(e_x).float()
        batch_size = e_x.shape[0]        
        if(batch_size != 1):
            print(batch_size)
            print("Batch size > 1 is not available now. Batch size please give 1.")
            return
        #print(sent.shape) torch.Size([2, 15, 6087])

        e_output, hiddens = self.encoder_RNN(e_x)
        
        #attention
        uW = self.trainable_W(e_output)
        
        first = True
        first_index = DIC_word_index['<bos>']
        first_input = torch.tensor([[first_index] * batch_size])
        first_input = first_input.view(batch_size, 1, 1)
        ans_seq = []
        for i in range(self.seq_length):
            #print(word.shape) torch.Size([2, 6087])
            if(first):
                word = first_input;
                first = False
                
            #print(word.shape)
            word = self.embedding(word.view(1, -1)).float();
            #print(word.shape)
            #one for each word, therefore d_output = d_state
            d_output, hiddens = self.decoder_RNN(word, hiddens)
            uWv = torch.matmul(uW, d_output.transpose(2,1))
            #print(uWv.shape) torch.Size([2, 80, 1])
            alpha = self.soft(uWv)
            #print(uWv)
            #print(alpha)
            #input("")
            new_context = e_output.transpose(2,1) @ alpha;
            #print(e_output.shape) torch.Size([2, 80, 256])
            #print(alpha.shape) torch.Size([2, 80, 1])
            
            #print(d_output.shape) torch.Size([2, 1, 256])
            #print(new_context.shape) torch.Size([2, 256, 1])          
            pre = torch.cat((d_output, new_context.transpose(2,1)), 2)
            ans_logits = self.Projection_layer(pre)
            ans_index = np.argmax(ans_logits.cpu().detach().numpy())
            ans = DIC_index_word[ans_index]
            ans_seq.append(ans)
            word = torch.tensor([ans_index])

        #logits = torch.stack(logits, 1)
        return ans_seq

In [5]:
#
def Inference(model_path, question_path):
    def pad_post_zero(a, length):
        ret = []
        for _list in a:
            if(len(_list) < length):
                for ct in range(len(_list),length,1):
                    _list.append(DIC_word_index["<pad>"])
            if(len(_list) > length):
                _list = _list[:length]

            ret.append(_list)
        return ret
    
    latent_dim = 1024
    embed_dim = word_vectors.shape[1]
    
    model = InferenceAttModel(latent_dim, sent_len, embed_dim, word_vectors)
    #saved_model = torch.load("models/Model92")
    saved_model = torch.load(model_path)
    model.load_state_dict(saved_model['model'])
    model = model.to(device)
    
    encode_x = []
    #with open('sel_conversation/question.txt', 'r') as f:
    with open(question_path, 'r') as f:
        for l in f:
            l = l.split()
            tmp = []
            for word in l:
                try:
                    index = DIC_word_index[word]
                except KeyError:
                    index = DIC_word_index['<unk>']
                tmp.append(index)
            encode_x.append([DIC_word_index['<bos>']] + tmp + [DIC_word_index['<eos>']])        
    encode_x = pad_post_zero(encode_x, sent_len)
    
    
    ANSWERS = list()
    for index in range(5):
        e_x = torch.tensor([encode_x[index]]).long().to(device)
        ans = model.forward(e_x)
        ANSWERS.append(ans)
    
    return(ANSWERS)
    

In [6]:
ANSWERS = Inference(model_path="models/Model92", question_path='sel_conversation/question.txt')

In [7]:
ANSWERS 

[['我們', '選', '了', '一個', '很'],
 ['來', '紀念', '退伍軍人', '節', '的'],
 ['今天', '早上', '<eos>', '會', '有'],
 ['就是', '保護', '我們', '了', '<eos>'],
 ['保護', '和', '權利', '而已', '<eos>']]